In [1]:
pip install tableaudocumentapi

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------- ----------------------------- 1.0/3.8 MB 20.5 MB/s eta 0:00:01
   ----------------------------------- ---- 3.3/3.8 MB 30.3 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 30.1 MB/s eta 0:00:00
  Created wheel for tableaudocumentapi: filename=tableaudocumentapi-0.11-py2.py3-none-any.whl size=16219 sha256=268df355d351a3a7f42c70caf2069d9a11484765e51ddc3f6d1e3f9956a0da03
  Stored in directory: c:\users\nuttapong.but\appdata\local\pip\cache\wheels\42\ce\57\c4df5bbdcfaa59d14a7c58aad6ed62c60222a6a63357878260
Successfully built tableaudocumentapi
Note: you may need


[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# เอาแค่ dic ออกมาจาก workbook เฉยๆ

from tableaudocumentapi import Workbook
import pandas as pd
import os

# Path to the Tableau workbook
tableau_workbook_path = r'D:\Tableau Hyper\LSP ติดตามค้างชำระ\LSP ติดตามค้างชำระ.twb'

# Load the Tableau workbook
workbook = Workbook(tableau_workbook_path)

# Extract workbook name for CSV file
workbook_base_name = os.path.splitext(os.path.basename(tableau_workbook_path))[0]
output_csv_file = os.path.join(os.path.dirname(tableau_workbook_path), workbook_base_name + '_DataDictionary.csv')

# Prepare data for DataFrame
data = []
for datasource in workbook.datasources:
    for field in datasource.fields.values():
        # Extract additional field properties
        field_role = getattr(field, 'role', 'N/A')
        field_calculation = getattr(field, 'calculation', 'N/A')
        field_caption = getattr(field, 'caption', 'N/A')
        field_description = getattr(field, 'description', 'N/A')

        data.append([
            datasource.name, 
            field.name, 
            field.datatype, 
            field_role, 
            field_calculation,
            field_caption,
            field_description
        ])

# Create DataFrame with more detailed columns
df = pd.DataFrame(data, columns=[
    'Datasource Name', 
    'Field Name', 
    'Field Type', 
    'Field Role', 
    'Field Calculation',
    'Field Caption',
    'Field Description'
])

# Optionally, export to CSV
df.to_csv(output_csv_file, index=False, encoding='utf-8-sig')
print(f"Data dictionary exported to {output_csv_file}")



Data dictionary exported to D:\Tableau Hyper\LSP ติดตามค้างชำระ\LSP ติดตามค้างชำระ_DataDictionary.csv


In [1]:
import openai
import pandas as pd
from tableaudocumentapi import Workbook
import os
import xml.etree.ElementTree as ET

# ฟังก์ชันสำหรับแทนที่ ID ด้วยชื่อฟิลด์
def replace_ids_with_names(calculation, field_id_to_name):
    """
    แทนที่ Field ID ด้วยชื่อ Field ในสตริงการคำนวณของ Tableau

    Parameters:
    calculation (str): สตริงการคำนวณที่มี Field ID
    field_id_to_name (dict): ดิกชันนารีที่มี Field ID เป็นคีย์และชื่อ Field เป็นค่า

    Returns:
    str: สตริงการคำนวณที่มีชื่อ Field แทนที่ Field ID
    """
    # ตรวจสอบว่ามีสตริงการคำนวณหรือไม่
    if calculation:
        # แสดงการคำนวณเดิม
        print("❌ การคำนวณเดิม:", calculation)
        # วนลูปเพื่อแทนที่ Field ID ด้วยชื่อ Field
        for field_id, field_name in field_id_to_name.items():
            # ตรวจสอบว่า Field ID มีอยู่ในสตริงการคำนวณหรือไม่
            if field_id in calculation:
                # ตรวจสอบว่าชื่อ Field มีวงเล็บอยู่แล้วหรือไม่
                replacement = f'[{field_name}]' if not field_name.startswith('[') else field_name
                # แสดงการแทนที่
                print(f"✔ กำลังแทนที่ {field_id} ด้วย 👉 {replacement}")
                # ทำการแทนที่ในสตริง
                calculation = calculation.replace(field_id, replacement)
        # แสดงการคำนวณหลังจากแทนที่
        print("✨ การคำนวณหลังการแทนที่:", calculation)
    return calculation


# ฟังก์ชันสำหรับใช้ AI ในการอนุมานคำอธิบาย จากคอลั่มอื่นๆ
def get_ai_description(row, api_key, model="gpt-3.5-turbo"): # เลือกได้ระหว่าง "gpt-3.5-turbo" ธรรมดา ไม่แพง ช้า, "gpt-4 turbo" ดี แพง ช้ามากๆๆ
    """
    สร้างคำอธิบายจาก AI สำหรับฟิลด์ใน Tableau โดยใช้ OpenAI

    Parameters:
    row (pd.Series): แถวใน DataFrame ที่มีข้อมูลฟิลด์
    api_key (str): API key สำหรับ OpenAI
    model (str): ชื่อโมเดล AI ที่ใช้ (เช่น "gpt-3.5-turbo")

    Returns:
    str: คำอธิบายที่สร้างโดย AI
    """
    try:
        openai.api_key = api_key
        #prompt = f"Please provide a brief description and confidence level  for the following Tableau field:\n- Field Name: {row['Field Name']}\n- Field Type: {row['Field Type']}\n- Field Role: {row['Field Role']}\n- Field Calculation: {row['Field Calculation']}\n\nDescription:"
        prompt = f"Please provide a concise Thai description and confidence level for the following Tableau field:\n- Field Name: {row['Field Name']}\n- Field Type: {row['Field Type']}\n- Field Role: {row['Field Role']}\n- Field Calculation: {row['Field Calculation Updated']}\n- Field Description: {row['Field Description']}\n- Default Aggregation: {row['Default Aggregation']}\n- Is Quantitative: {row['Is Quantitative']}\n- Is Ordinal: {row['Is Ordinal']}\n- Is Nominal: {row['Is Nominal']}\n\nDescription:"

        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"Error generating description: {e}"    



def export_enhanced_tableau_data_dictionary(tableau_workbook_path, api_key=None, model="gpt-3.5-turbo", include_ai_description=False):
    """
    ส่งออก data dictionary ขั้นสูงจาก Tableau workbook ไปยังไฟล์ CSV พร้อมทางเลือกในการใช้ AI สำหรับคำอธิบาย

    Parameters:
    tableau_workbook_path (str): ที่อยู่ไฟล์ของ Tableau workbook
    api_key (str): API key สำหรับ OpenAI (ถ้าต้องการใช้ AI)
    model (str): ชื่อโมเดล AI ที่ใช้ (เช่น "gpt-3.5-turbo" หรือ "gpt-4 turbo")
    include_ai_description (bool): ตัวเลือกในการเพิ่มคำอธิบายจาก AI

    Returns:
    tuple: DataFrame ที่มีข้อมูล data dictionary และที่อยู่ไฟล์ CSV ที่ถูกส่งออก
    """
    # โหลด Workbook และประมวลผลข้อมูล
    workbook = Workbook(tableau_workbook_path)

    # กำหนดชื่อไฟล์ CSV สำหรับส่งออก
    workbook_base_name = os.path.splitext(os.path.basename(tableau_workbook_path))[0]
    output_csv_file = os.path.join(os.path.dirname(tableau_workbook_path), workbook_base_name + '_EnhancedDataDictionary.csv')

    # สร้างลิสต์เพื่อเก็บข้อมูลของแต่ละ field
    data = []
    # วนลูปเข้าถึงแต่ละ datasource และ field ใน workbook
    for datasource in workbook.datasources:
        for field_id, field in datasource.fields.items():
            field_description_xml = getattr(field, 'description', None)
            field_description = ''
            # แก้ไขให้ field_description แสดงแต่คำอธิบาย
            if field_description_xml is not None:
                try:
                    root = ET.fromstring(field_description_xml)
                    extracted_text = ''.join(root.itertext())
                    field_description = ' '.join(extracted_text.split())
                except ET.ParseError:
                    field_description = field_description_xml
    
            # เก็บคุณสมบัติของ field
            field_properties = {
                'Datasource Name': datasource.name,
                'Field Name': field.name,
                'Field ID': field_id,
                'Field Type': field.datatype,
                'Field Role': getattr(field, 'role', ''),
                'Field Calculation': getattr(field, 'calculation', ''),
                'Field Caption': getattr(field, 'caption', ''),
                'Field Description': field_description,
                'Default Aggregation': getattr(field, 'default_aggregation', ''),
                'Is Quantitative': getattr(field, 'is_quantitative', ''),
                'Is Ordinal': getattr(field, 'is_ordinal', ''),
                'Is Nominal': getattr(field, 'is_nominal', '')
            }
            data.append(field_properties)

    # สร้าง DataFrame จากข้อมูลที่เก็บไว้
    df = pd.DataFrame(data)
    # สร้างดิกชันนารีเพื่อแมป Field ID ไปยังชื่อ Field
    field_id_to_name = {field['Field ID']: field['Field Name'] for field in data}
    # ปรับปรุงคอลัมน์ 'Field Calculation' โดยการเรียกใช้ฟังก์ชัน replace_ids_with_names
    df['Field Calculation Updated'] = df.apply(
        lambda row: replace_ids_with_names(row['Field Calculation'], field_id_to_name), axis=1
    )

    # ถ้าเลือกใช้ AI ในการสร้างคำอธิบาย
    if include_ai_description and api_key:
        df['AI Description'] = df.apply(lambda row: get_ai_description(row, api_key, model), axis=1)

    # ส่งออก DataFrame เป็นไฟล์ CSV
    df.to_csv(output_csv_file, index=False, encoding='utf-8-sig')
    print(f"ได้ส่งออก data dictionary ขั้นสูงไปที่ {output_csv_file}")

    return df, output_csv_file


# ตัวอย่างการใช้งาน
api_key = "replace with your api key"  # ใส่ API key ของ OpenAI แทนที่ "your-openai-api-key"
model="gpt-4-turbo"  # เลือกใช้ระหว่าง "gpt-3.5-turbo" ธรรมดา ถูก เร็ว, "gpt-4-turbo" ดีกว่า แพงกว่า 75 เท่า ช้ากว่า 10 เท่า

df_sample, output_csv_path = export_enhanced_tableau_data_dictionary(
    r'D:\Tableau Hyper\Human Resource\People Effectiveness Dashboard\People Effectiveness Dashboard.twbx',  # ชี้ path ไปที่ tableau workbook ที่เราต้องการ
    api_key=api_key,
    model=model,
    include_ai_description=True # เปิดหรือปิด AI ด้วยการกำหนดค่า True หรือ False
)


❌ การคำนวณเดิม: "Segment"
✨ การคำนวณหลังการแทนที่: "Segment"
❌ การคำนวณเดิม: "Revenue per headcount"
✨ การคำนวณหลังการแทนที่: "Revenue per headcount"
❌ การคำนวณเดิม: "Staff Cost per headcount"
✨ การคำนวณหลังการแทนที่: "Staff Cost per headcount"
❌ การคำนวณเดิม: "All"
✨ การคำนวณหลังการแทนที่: "All"
❌ การคำนวณเดิม: "2024-04"
✨ การคำนวณหลังการแทนที่: "2024-04"
❌ การคำนวณเดิม: //อัตราส่วนค่าเสื่อมราคาที่เกิดจากสินทรัพย์ตามสัญญาเช่า ต่อ ยอดขาย
[Calculation_761952821463281667] / [Calculation_704250456195362817]
✔ กำลังแทนที่ [Calculation_704250456195362817] ด้วย 👉 [Sales]
✔ กำลังแทนที่ [Calculation_761952821463281667] ด้วย 👉 [Depreciation and Amortization-ROU]
✨ การคำนวณหลังการแทนที่: //อัตราส่วนค่าเสื่อมราคาที่เกิดจากสินทรัพย์ตามสัญญาเช่า ต่อ ยอดขาย
[Depreciation and Amortization-ROU] / [Sales]
❌ การคำนวณเดิม: //กำไรส่วนเกินที่จะมาชดเชยค่าใช้จ่ายคงที่ หรือก็คือรายได้หลังหักค่าใช้จ่ายผันแปร
[Calculation_761952821478039559] - [Rental (copy)_1096626570862583812]
✔ กำลังแทนที่ [Calculation_76195